In [214]:
#import packages
import numpy as np
import cv2
cv2.startWindowThread()


0

In [215]:
#open video
cap = cv2.VideoCapture('/Users/danielrosenthal/Documents/EGR_314/Daniel_7_top_trim.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) #get fps

if cap.isOpened() == False:
    print("Cannot open video file.")

In [216]:
#saving output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
#print(size)
output = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))
output2 = cv2.VideoWriter('output2.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 20.0, (1920,1080))

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [217]:
#object detection parameters
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold= 90)

In [218]:
#creating empty array
arr = np.empty((0,4))

#defining rgb range

lower_color_bound = np.array([0,100,178])
upper_color_bound = np.array([255,255,255])


In [219]:
while True:
    ret, frame = cap.read()
    
    #region of interest
    height, width, _ = frame.shape
    roi = frame[200:900, 100:1800]
    
    #object detection
    mask = object_detector.apply(roi)

    #cleaning mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #trying rgb threshold
    mask = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV) 
    mask_rgb = cv2.inRange(mask,lower_color_bound,upper_color_bound)
    rgb_contours, _ = cv2.findContours(mask_rgb, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       
    
    for cnt in contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            # cv2.drawContours(roi, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            if (x > 200 and x < 900 and y > 100 and y < 1800):
                 #print(x, y, (x+w), (y+h))
                 np.append(arr, (x, y, x + w, y + h))
                 #print(arr)
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0,255,0),3)

    for cnt in rgb_contours:
        #calculate error, remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            cv2.drawContours(mask_rgb, [cnt], -1, (0,255,0), 2)
            #drawing rectangle around detected objects
            x, y, w, h = cv2.boundingRect(cnt)
            if (x > 200 and x < 900 and y > 100 and y < 1800):
                 #print(x, y, (x+w), (y+h))
                 np.append(arr, (x, y, x + w, y + h))
                 #print(arr)
            cv2.rectangle(mask_rgb, (x, y), (x + w, y + h), (0,255,0),3)

    cv2.imshow("roi", roi)
    #cv2.imshow("Frame",frame)
    cv2.imshow("Mask", mask)
    #cv2.imshow("blobs", blobs)
    cv2.imshow("maskrgb",mask_rgb)
    
    output.write(frame)
    #output2.write(mask)
    if cv2.waitKey(120) & 0xFF == ord('q'):
        break

#print(height, width)
cap.release()
output.release()
output2.release()
cv2.destroyAllWindows()
#cv2.waitKey(1)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
print(arr)

[]
